In [134]:
from PIL import Image
import numpy as np
import os
import cv2

In [135]:
data=[]
labels=[]
cats=os.listdir("Dataset/cats")
for cat in cats:
    imag=cv2.imread("Dataset/cats/"+cat)
    ArrayImages = Image.fromarray(imag, 'RGB')
    resized_image = ArrayImages.resize((200, 200))
    data.append(np.array(resized_image))
    labels.append(0)
dogs=os.listdir("Dataset/dogs/")
for dog in dogs:
    imag=cv2.imread("Dataset/dogs/"+dog)
    ArrayImages = Image.fromarray(imag, 'RGB')
    resized_image = ArrayImages.resize((200, 200))
    data.append(np.array(resized_image))
    labels.append(1)
    
humans=os.listdir("Dataset/Humans")
for human in humans:
    imag=cv2.imread("Dataset/humans/"+human)
    ArrayImages = Image.fromarray(imag, 'RGB')
    resized_image = ArrayImages.resize((200, 200))
    data.append(np.array(resized_image))
    labels.append(2)
horses=os.listdir("Dataset/horses")
for horse in horses:
    imag=cv2.imread("Dataset/horses/"+horse)
    ArrayImages = Image.fromarray(imag, 'RGB')
    resized_image = ArrayImages.resize((200, 200))
    data.append(np.array(resized_image))
    labels.append(3)

In [136]:
Races=np.array(data)
labels=np.array(labels)

In [137]:
np.save("Races",Races)
np.save("labels",labels)

In [138]:
Races=np.load("Races.npy")
labels=np.load("labels.npy")

In [139]:
s=np.arange(Races.shape[0])
np.random.shuffle(s)
Races=Races[s]
labels=labels[s]

In [140]:
num_classes=len(np.unique(labels))
data_length=len(Races)

In [141]:
(x_train,x_test)=Races[(int)(0.1*data_length):],Races[:(int)(0.1*data_length)]
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255
train_length=len(x_train)
test_length=len(x_test)

In [142]:
(y_train,y_test)=labels[(int)(0.1*data_length):],labels[:(int)(0.1*data_length)]

In [143]:
import keras
from keras.utils import np_utils
y_train=keras.utils.to_categorical(y_train,num_classes)
y_test=keras.utils.to_categorical(y_test,num_classes)

In [144]:
# import sequential model and all the required layers
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
#make model
model=Sequential()
model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(200,200,3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(500,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(4,activation="softmax"))
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_23 (Conv2D)          (None, 200, 200, 16)      208       
                                                                 
 max_pooling2d_23 (MaxPoolin  (None, 100, 100, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_24 (Conv2D)          (None, 100, 100, 32)      2080      
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 50, 50, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_25 (Conv2D)          (None, 50, 50, 64)        8256      
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 25, 25, 64)      

In [145]:
# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', 
                  metrics=['accuracy'])

In [146]:
model.fit(x_train,y_train,batch_size=50
          ,epochs=100,verbose=1)

Epoch 1/100
15/15 [==============================] - 12s 655ms/step - loss: 3.4349 - accuracy: 0.2514
Epoch 2/100
15/15 [==============================] - 10s 689ms/step - loss: 1.3326 - accuracy: 0.3448
Epoch 3/100
15/15 [==============================] - 12s 781ms/step - loss: 1.1530 - accuracy: 0.5371
Epoch 4/100
15/15 [==============================] - 10s 649ms/step - loss: 0.8809 - accuracy: 0.6456
Epoch 5/100
15/15 [==============================] - 10s 639ms/step - loss: 0.6799 - accuracy: 0.7445
Epoch 6/100
15/15 [==============================] - 10s 643ms/step - loss: 0.5276 - accuracy: 0.8187
Epoch 7/100
15/15 [==============================] - 10s 657ms/step - loss: 0.3384 - accuracy: 0.8874
Epoch 8/100
15/15 [==============================] - 10s 669ms/step - loss: 0.2643 - accuracy: 0.9135
Epoch 9/100
15/15 [==============================] - 10s 639ms/step - loss: 0.1845 - accuracy: 0.9464
Epoch 10/100
15/15 [==============================] - 10s 638ms/step - loss: 0.142

In [147]:
score = model.evaluate(x_test, y_test, verbose=1)
print('\n', 'Test accuracy:', score[1])

3/3 [==============================] - 0s 68ms/step - loss: 2.3012 - accuracy: 0.6875

 Test accuracy: 0.6875


In [180]:
def convert_to_array(img):
    im = cv2.imread(img)
    img = Image.fromarray(im, 'RGB')
    image = img.resize((200, 200))
    return np.array(image)
def get_Race_name(label):
    if label==0:
        return "cat"
    if label==1:
        return "dog"
    if label==2:
        return "human"
    if label==3:
        return "horse"
def predict_Race(file):
    ar=convert_to_array(file)
    ar=ar/255
    label=1
    a=[]
    a.append(ar)
    a=np.array(a)
    score=model.predict(a,verbose=1)
    print(score)
    label_index=np.argmax(score)
    print(label_index)
    acc=np.max(score)
    Race=get_Race_name(label_index)
    print(Race)
    print("The predicted Race is a "+Race+" with accuracy =    "+str(acc))

In [181]:
test_images =os.listdir("TestPictures/")
for image in test_images:
    print('Picture name = '+ image)
    print("#########################################################################")
    predict_Race("TestPictures/"+image)
    print("#########################################################################")


1/1 [==============================] - 0s 40ms/step
[[9.9905092e-01 9.2962105e-04 5.1725732e-07 1.8934557e-05]]
0
cat
The predicted Race is a cat with accuracy =    0.9990509
1/1 [==============================] - 0s 35ms/step
[[9.9972004e-01 2.7734452e-04 2.0636357e-06 5.5511714e-07]]
0
cat
The predicted Race is a cat with accuracy =    0.99972004
1/1 [==============================] - 0s 35ms/step
[[9.99767363e-01 2.30281687e-04 1.00821405e-08 2.33332594e-06]]
0
cat
The predicted Race is a cat with accuracy =    0.99976736
1/1 [==============================] - 0s 33ms/step
[[8.54100927e-06 9.99991417e-01 1.34619995e-08 4.68465906e-08]]
1
dog
The predicted Race is a dog with accuracy =    0.9999914
1/1 [==============================] - 0s 34ms/step
[[7.9684020e-10 1.0000000e+00 2.8716725e-09 5.9592764e-10]]
1
dog
The predicted Race is a dog with accuracy =    1.0
1/1 [==============================] - 0s 35ms/step
[[9.9990070e-01 9.7874734e-05 1.2635384e-06 8.8687074e-08]]
0
cat
The